In [1]:
import numpy as np
import re
import pandas as pd
import os

coord_re = r'\s*([CHONF])\s+(-?\d+\.\d+)\s+(-?\d+\.\d+)\s+(-?\d+\.\d+)'

num_atoms = 9

In [2]:
U_worksheet = pd.read_csv('rotation/U0.csv')
def get_dihe_U0(name):
    U0 = U_worksheet[U_worksheet['Name'] == name]['U0'].values[0]
    dihe = U_worksheet[U_worksheet['Name'] == name]['dihedral'].values[0]
    return float(U0), float(dihe)

In [3]:
def get_element_coord(lines):
    elements = []
    ele2idx = {'H': 1, 'C': 6, 'N': 7, 'O': 8, 'F':9}
    coords = []
    for line in lines:
        m = re.match(coord_re, line)
        if m:
            elements.append(ele2idx[m.group(1)])
            coords.append([float(m.group(i)) for i in range(2, 5)])
    return elements, np.array(coords)

In [4]:
root_dir = 'rotation/'

elements_list = []
coords_list = []
U0_list = []
dihe_list = []

for idx in range(1,38):
    bn = 'rotation_std'+str(idx)
    filename = os.path.join(root_dir, bn+'.gjf')
    with open(filename, "r") as f:
        lines = f.readlines()
        elements, coords = get_element_coord(lines)
        # print(len(elements), len(coords))

    U0, dihe = get_dihe_U0(bn)
    
    elements_list = elements_list + elements
    coords_list = coords_list + coords.tolist()
    U0_list.append(U0)
    dihe_list.append(dihe)

In [5]:
N_list = [num_atoms] * len(U0_list)

N_list = np.array(N_list)
elements_list = np.array(elements_list)
coords_list = np.array(coords_list)
U0_list = np.array(U0_list)
dihe_list = np.array(dihe_list)

np.savez('rotation.npz', N=N_list, Z=elements_list, R=coords_list, U0=U0_list, D=dihe_list)